<a href="https://colab.research.google.com/github/MaleNurse/YOLOv5_Trainer/blob/main/YOLOv5_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup environment

Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Update environment

In [ ]:
! apt update
! apt upgrade
! apt install libcudnn7 libcublas-dev libcublas10 libcudnn7 libcudnn7-dev libnccl-dev libnccl2 --allow-change-held-packages
! echo "Segment complete"

Check GPU and Driver Version (req >=460)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Install YOLOv5 and depends

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install imgaug==0.2.5  # deal w/ dependency issue
!pip install -U -r yolov5/requirements.txt  # install dependencies
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html # update torch to work with newer drivers
!pip install wandb
!echo "Segment complete RESTART"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"
!echo "RESTART THE RUNTIME"

Update YOLOv5 if, for some reason, it's already cloned

In [ ]:
%cd /content/yolov5
!git pull

Check Torch environment (ensure no errors thrown)

In [ ]:
%cd /content/yolov5
import torch
from IPython.display import Image  # for displaying images
from utils.google_utils import gdrive_download  # for downloading models/datasets

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Customize iPython writefile so we can write variables

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))
!echo "Segment complete"

# Setup training model

Define training parameters and make dependant paths/strings

**Known good configs**

yolov5x batch 18 img 640 epoch 250 w/ deepstack mod

yolov5l batch 20 img 736 epoch ??? w/ deepstack mod

In [ ]:
# Set variables above separator. Don't delete the quote marks!

# Known good configs
# yolov5x batch 18 img 640 epoch 250 
# yolov5l batch 20 img 736 epoch 150 w/ checkpoint

model_size = 'l'        # trailing end of the training model desired (s, m, l, or x)
check = 'y'             # use a checkpoint as a starting point in the training? ('y' if yes)
deepstack = 'y'         # training for deepstack server? ('y' is yes)
img_size = '736'        # to what resolution to scale training images (e.g. 640) must be mult of 32
batch_size = '20'
epochs = '150'
comment = 'aug3'        # something to distringuish the run name, if needed (e.g. aug_dsx)
test_img_size = '736'   # to what resolution to scale testing images (e.g. 960)  must be mult of 32
test_conf = '0.5'       # confidence threshold for inference testing (e.g. 0.5)

##########################################################################################

import os.path, shutil, os
from os import path

i = 1
j = 1

#make base model path
model_yaml = '/content/yolov5/models/yolov5' + model_size + '.yaml'

if not comment: 
  #make run name if no comment entered
  run_name = '5' + model_size + '_i' + img_size + '_b' + batch_size
else:
  #make run name if comment exists
  run_name = '5' + model_size + '_i' + img_size + '_b' + batch_size + '_' + comment

if deepstack == 'y':
  run_name = run_name + '_ds'

#if directory exists with current run name, start incrementing run name by 1 until unique
if path.exists(path.join('/content/drive/MyDrive/colab_out/', run_name)):                 
  while path.exists(path.join('/content/drive/MyDrive/colab_out/', run_name + str(i))):
    i += 1
  run_name = run_name + str(i)

#set path to run
run_path = '/content/drive/MyDrive/colab_out/' + run_name

exp_dir = '/content/drive/MyDrive/colab_out/exp'

#find next available exp directory if "exp" already exists
if path.exists('/content/drive/MyDrive/colab_out/exp'):                 
  while path.exists(path.join('/content/drive/MyDrive/colab_out/exp' + str(j))):
    j += 1
  exp_dir = exp_dir + str(j)

#make string for inference testing output
exp_img = path.join(exp_dir, '*.jpg')

#set path to weights
weights = run_path + '/weights/best.pt'

#set checkpoint name
if check == 'y':
  chkpnt = 'yolov5' + model_size + '.pt'
  #if training for deepstack, copy v3 weight file to correct path so v4 isn't downloaded
  if deepstack == 'y'
    chkpnt_tmp = path.join('/content/drive/MyDrive/v3_weights', chkpnt)
    shutil.copy(chkpnt_tmp, '/content/yolov5/')
else:
  chkpnt = ''

!echo "This run is called $run_name and the exp folder will be $exp_dir"

In [ ]:
import os.path, shutil, os
from os import path

shutil.copy(chkpnt_tmp, '/content/yolov5/')

Define number of classes based on YAML from Roboflow

In [ ]:
%cd /content/drive/MyDrive/my-dataset
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
!echo "Segment complete"

Modify Roboflow YAML with correct directories

In [ ]:
%cd /content/drive/MyDrive/my-dataset
!sed 's/\.\./\/content\/drive\/MyDrive\/my-dataset/' data.yaml >temp.yaml
!mv temp.yaml data.yaml
!echo "Segment complete"

Output desired model size (adjust as necessary)

In [ ]:
%cat $model_yaml

Create custom model using exact class size.

Copy the cat output above starting at depth_multiple and replace the same below

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, C3, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

Adjust model if training for DeepStack

In [ ]:
%cd /content/yolov5/models/
!sed 's/C3/BottleneckCSP/' custom_yolov5.yaml >temp.yaml
!mv temp.yaml custom_yolov5.yaml
!echo "Segment complete"

Make sure the custom model looks OK (and exists)

In [ ]:
!cat /content/yolov5/models/custom_yolov5.yaml

# Train!

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --batch $batch_size --img-size $img_size --epochs $epochs --data '/content/drive/MyDrive/my-dataset/data.yaml' --cfg ./models/custom_yolov5.yaml --weights '$chkpnt' --name $run_name  --project /content/drive/MyDrive/colab_out/ --cache-images

# Set variables if environment lost in a runtime disconnection

Make sure to rerun environment setup, too

In [ ]:
run_name = '5x_i640_b18_nopre_ds' # look for first two in /content/drive/MyDrive/colab_out/
exp_num = ''                   # leave as '' if just exp
test_img_size = '1280'           # to what resolution to scale testing images (e.g. 960) must be mult of 32
test_conf = '0.5'               # confidence threshold for inference testing (e.g. 0.5)

#############################################################################

import os.path
from os import path

run_path = '/content/drive/MyDrive/colab_out/' + run_name
weights = run_path + '/weights/best.pt'
exp_img = path.join('/content/drive/MyDrive/colab_out/exp' + exp_num, '*.jpg')

# Evaluate training

Start tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $run_path

# Test completed weights

Inference speed test

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
%cd /content/yolov5/
!python detect.py --weights $weights --img $test_img_size --conf $test_conf --source /content/drive/MyDrive/my-dataset/test/images --project /content/drive/MyDrive/colab_out/

Display inference on all test images

In [ ]:
import glob
import os.path
from os import path
from IPython.display import Image, display

for imageName in glob.glob(exp_img): #assuming JPG
    display(Image(filename=imageName))
    print("\n")